In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
data=pd.read_csv('../data/data.csv')

In [3]:
data.shape

(2099998, 5)

In [4]:
data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
# print(data[data['label']!=-1].groupby('query_len')['label'].mean())
# print('====================')
data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('prefix_len')['label'].mean())
# print('====================')
# data['tag_len']=data['tag'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('tag_len')['label'].mean())
# print('====================')
data['title_len']=data['title'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('title_len')['label'].mean())
# print('====================')
# print(data.shape)
# print('====================')
# print(data.columns)
# print('====================')
# print(data.describe())
# print('====================')
# print(data.info())

In [5]:
gc.collect()

21

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lr_col=['prefix','title','tag','query']
for feature in lr_col:
    data[feature]=data[feature].astype(str)

data['prefix']=le.fit_transform(data['prefix'])
data['title']=le.fit_transform(data['title'])
data['query']=le.fit_transform(data['query'])
data['tag']=le.fit_transform(data['tag'])

In [7]:
traindf=data[data['label']!=-1]
testdf=data[data['label']==-1]
print(traindf.shape)
print(testdf.shape)

(2049998, 8)
(50000, 8)


In [8]:
col=['prefix','query','title','tag','query_len','prefix_len','title_len']

for item in col:
    temp=traindf.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
    temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
    traindf = pd.merge(traindf, temp, on=item, how='left')
    testdf=pd.merge(testdf,temp,on=item,how='left')
    #testdf中有很多是traindf中没有的，所以导致testdf.merger的时候就会出现很多NaN

In [9]:
for i in range(len(col)):
    for j in range(i+1,len(col)):
        item_g=[col[i],col[j]]
        temp=traindf.groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'_count':'count'})
        temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'_count']+3)
        traindf=pd.merge(traindf,temp,on=item_g,how='left')
        testdf=pd.merge(testdf,temp,on=item_g,how='left')

In [10]:
print(traindf.shape,testdf.shape)

(2049998, 92) (50000, 92)


# ===========Train===============

In [11]:
# traindf.to_csv('../data/final_train2.csv',index=False)
testddf=testdf.drop(['label'],axis=1)
# testddf.to_csv('../data/final_test2.csv',index=False)

In [12]:
y=traindf.pop('label').values
X=traindf.as_matrix()
testX=testddf.as_matrix()

In [13]:
from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [14]:
sub_train=pd.DataFrame()
sub_test=pd.DataFrame()

In [15]:
losstrain=0
lossval=0
fs=0
ft=0

N=10

skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)

clf= LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=-1, 
                         learning_rate=0.05, n_estimators=2000,objective='binary', 
                         min_split_gain=0,min_child_weight=5, min_child_samples=10, 
                         subsample=0.8, subsample_freq=1,colsample_bytree=0.8, 
                         reg_alpha=3, reg_lambda=5, random_state=1, n_jobs=-1)

In [16]:
for i,(train_index,test_index) in enumerate(skf.split(X,y)):
    print("Fold=%s"%(i))
    model=clf.fit(X[train_index],y[train_index],
                 eval_names=['train','valid'],
                 eval_metric='logloss',
                  eval_set=[(X[train_index],y[train_index]),(X[test_index],y[test_index])],
                 early_stopping_rounds=100,
                  verbose=50,)
    
    trainss=model.best_score_['train']['binary_logloss']
    valss=model.best_score_['valid']['binary_logloss']
    losstrain+=trainss
    lossval+=valss
    print("best logloss on train_set:",trainss)
    print("best logloss on test_set:",valss)
    
    train_proba=model.predict_proba(X[train_index],num_iteration=model.best_iteration_)[:,1]
    train_pred=np.where(train_proba>0.37,1,0)
    ftrain=f1_score(y[train_index],train_pred)
    print("f1_score on train_set:",ftrain)
    ft+=ftrain
    
    val_proba=model.predict_proba(X[test_index],num_iteration=model.best_iteration_)[:,1]
    val_pred=np.where(val_proba>0.37,1,0)
    fval=f1_score(y[test_index],val_pred)
    print("f1_score on test_set:",fval)
    fs+=fval
    
    train_all=model.predict_proba(X,num_iteration=model.best_iteration_)[:,1]
    sub_train['ans_%s'%str(i)]=train_all
    test_proba=model.predict_proba(testX,num_iteration=model.best_iteration_)[:,1]
    sub_test['ans_%s'%str(i)]=test_proba
print("mean logloss in train set：%.5f"%(losstrain/N))
print("mean logloss in test set：%.5f"%(lossval/N))
print("mean f1 in train set：%.5f"%(ft/N))
print("mean f1 in test set：%.5f"%(fs/N))

Fold=0
Training until validation scores don't improve for 100 rounds.
[50]	train's binary_logloss: 0.336964	valid's binary_logloss: 0.337109
[100]	train's binary_logloss: 0.322936	valid's binary_logloss: 0.323395
[150]	train's binary_logloss: 0.321467	valid's binary_logloss: 0.322248
[200]	train's binary_logloss: 0.321086	valid's binary_logloss: 0.322239
[250]	train's binary_logloss: 0.320886	valid's binary_logloss: 0.322464
Early stopping, best iteration is:
[170]	train's binary_logloss: 0.321267	valid's binary_logloss: 0.322185
best logloss on train_set: 0.3212672203670879
best logloss on test_set: 0.32218456552549257
f1_score on train_set: 0.8009451469807038
f1_score on test_set: 0.7995975305268127
Fold=1
Training until validation scores don't improve for 100 rounds.
[50]	train's binary_logloss: 0.337016	valid's binary_logloss: 0.336467
[100]	train's binary_logloss: 0.323057	valid's binary_logloss: 0.322658
[150]	train's binary_logloss: 0.321575	valid's binary_logloss: 0.321448
[200

In [17]:
sub_train['label']=0
for i in range(N):
    sub_train['label']+=sub_train['ans_%s'%str(i)]
sub_train['label']=sub_train['label']/N

In [18]:
sub_test['label']=0
for i in range(N):
    sub_test['label']+=sub_test['ans_%s'%str(i)]
sub_test['label']=sub_test['label']/N

In [19]:
print(sub_train.shape)

(2049998, 11)


In [20]:
print(sub_test.shape)

(50000, 11)


In [21]:
sub_train['label'].to_csv('../data/ensemble/lgb_train.csv',index=True)
sub_test['label'].to_csv('../data/ensemble/lgb_test.csv',index=True)

# ==============ensemble===============

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [2]:
lgb_train=pd.read_csv('../data/ensemble/lgb_train.csv', names= ['lgb_label'], header= None)
lgb_test=pd.read_csv('../data/ensemble/lgb_test.csv',names= ['lgb_label'], header= None)

In [3]:
lr_train=pd.read_csv('../data/ensemble/lr_trainn.csv')
lr_train.columns=['index','lr_label']
lr_test=pd.read_csv('../data/ensemble/lr_testt.csv')
lr_test.columns=['index','lr_label']

In [4]:
print(lgb_train.shape,lgb_test.shape,lr_train.shape,lr_test.shape)

(2049998, 1) (50000, 1) (2049998, 2) (50000, 2)


In [5]:
lr_train=lr_train.drop(['index'],axis=1)
lr_test=lr_test.drop(['index'],axis=1)

In [6]:
data=pd.read_csv('../data/data.csv')

In [7]:
ntrain=pd.concat([lgb_train,lr_train],axis=1)
ntrain['label']=data['label'][data['label']!=-1]
print(ntrain.shape)
print(ntrain.columns)
print(ntrain['label'].value_counts())

(2049998, 3)
Index(['lgb_label', 'lr_label', 'label'], dtype='object')
0    1287218
1     762780
Name: label, dtype: int64


In [8]:
print(762780/2049998)

0.3720881678908955


In [9]:
ntest=pd.concat([lgb_test,lr_test],axis=1)
print(ntest.shape)
print(ntest.columns)

(50000, 2)
Index(['lgb_label', 'lr_label'], dtype='object')


In [10]:
lt=0
lv=0
fin=pd.DataFrame()

In [11]:
Y=ntrain.pop('label').values
x=ntrain.as_matrix()
testx=ntest.as_matrix()

In [13]:
M=10
skf=StratifiedKFold(n_splits=M,random_state=1,shuffle=True)
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(penalty='l2', dual=False, tol=0.001, C=1, random_state=1,
                            solver='liblinear', max_iter=100)

In [20]:
for i,(train_index,test_index) in enumerate(skf.split(x,Y)):
    print("Fold=%s"%str(i))
    clf.fit(x[train_index],Y[train_index])
    
    ytrain=clf.predict_proba(x[train_index])[:,1]
    ft=f1_score(Y[train_index],np.where(ytrain>0.37,1,0))
    print('f1 score on train set:%.5f'%ft)
    
    yval=clf.predict_proba(x[test_index])[:,1]
    fv=f1_score(Y[test_index],np.where(yval>0.37,1,0))
    print('f1 score on val set:%.5f'%fv)
    
    ytest=clf.predict_proba(testx)[:,1]
    fin['ans_%s'%str(i)]=ytest

Fold=0
[LibLinear]f1 score on train set:0.80092
f1 score on val set:0.80070
Fold=1
[LibLinear]f1 score on train set:0.80074
f1 score on val set:0.80216
Fold=2
[LibLinear]f1 score on train set:0.80100
f1 score on val set:0.79993
Fold=3
[LibLinear]f1 score on train set:0.80100
f1 score on val set:0.80006
Fold=4
[LibLinear]f1 score on train set:0.80093
f1 score on val set:0.80031
Fold=5
[LibLinear]f1 score on train set:0.80101
f1 score on val set:0.79983
Fold=6
[LibLinear]f1 score on train set:0.80066
f1 score on val set:0.80288
Fold=7
[LibLinear]f1 score on train set:0.80096
f1 score on val set:0.80021
Fold=8
[LibLinear]f1 score on train set:0.80088
f1 score on val set:0.80120
Fold=9
[LibLinear]f1 score on train set:0.80078
f1 score on val set:0.80167


In [21]:
fin['label']=0
for i in range(M):
    fin['label']+=fin['ans_%s'%str(i)]
fin['label']=fin['label']/M

In [22]:
fin['label']=fin['label'].apply(lambda x:np.where(x>0.37,1,0))
print(fin['label'].value_counts())

0    33224
1    16776
Name: label, dtype: int64


In [23]:
print(16776/50000)

0.33552


In [24]:
fin['label'].to_csv('../data/BaselineV1_4.csv',index=False,header=False)

# ==============Weighting===============

In [ ]:
ntest['nlabel']=ntest['lgb_label']*2+ntest['lr_label']*1
ntest['nlabel']=ntest['nlabel']/3
ntest['nlabel']=ntest['nlabel'].apply(lambda x:np.where(x>0.37,1,0))
print(ntest['nlabel'].value_counts())

In [ ]:
print(17800/50000)

In [ ]:
ntest['nlabel'].to_csv('../data/BaselineV1_42.csv',index=False,header=False)

In [ ]:
100/2971==0.7924